<a href="https://colab.research.google.com/github/serenach33/aiffel_ex/blob/main/E1_RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration 1: Rock-Paper-Scissors Image Classification

---

Model
* Keras Sequential API

Datasets
* Created my own datasets from [Teachable Machine](https://teachablemachine.withgoogle.com/)
* Gathered more datasets from Aiffel mates

Model evaluation
* Loss and Accuracy
* Confusion Matrix
* Classification report
